In [1]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

In [2]:
bs_path = "Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_937/Processing/20230101_air_937_business_shell_v31_baseline_results_w20230711-071622.csv"
test1_path = "Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_937/Processing/20230101_air_937_business_shell_v31_test1_results_w20230711-092250.csv"
test2_path = "Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_937/Processing/20230101_air_937_business_shell_v31_test2_results_w20230711-113825.csv"

In [3]:
baseline = pd.read_adls(bs_path, reader = pd.read_csv, encoding='iso-8859-1')
print(baseline.shape)
test_1 = pd.read_adls(test1_path, reader = pd.read_csv, encoding='iso-8859-1')
print(test_1.shape)
test_2 = pd.read_adls(test2_path, reader = pd.read_csv, encoding='iso-8859-1')
print(test_2.shape)

/anaconda/envs/data-science/lib/python3.10/site-packages/dsgtools/azure/adls.py:235: DtypeWarning: Columns (111,130,2302) have mixed types. Specify dtype option on import or set low_memory=False.
  data = reader(file, **kwargs)


(99939, 2304)


/anaconda/envs/data-science/lib/python3.10/site-packages/dsgtools/azure/adls.py:235: DtypeWarning: Columns (111,130,2302) have mixed types. Specify dtype option on import or set low_memory=False.
  data = reader(file, **kwargs)


(99998, 2304)


/anaconda/envs/data-science/lib/python3.10/site-packages/dsgtools/azure/adls.py:235: DtypeWarning: Columns (111,130,2302) have mixed types. Specify dtype option on import or set low_memory=False.
  data = reader(file, **kwargs)


(100000, 2304)


In [4]:
print(len([x for x in baseline.accountnumber if "SBFE" in x]))
print(len([x for x in test_1.accountnumber if "SBFE" in x]))
print(len([x for x in test_2.accountnumber if "SBFE" in x]))

49986
49998
50000


In [5]:
baseline["id"] = baseline.accountnumber + "_" + baseline.bus_company_name.fillna("").replace(' ', '_', regex=True)
test_1["id"] = test_1.accountnumber + "_" + test_1.bus_company_name.fillna("").replace(' ', '_', regex=True)
test_2["id"] = test_2.accountnumber + "_" + test_2.bus_company_name.fillna("").replace(' ', '_', regex=True)

In [6]:
baseline[['liennewesttype', 'judgmentnewesttype', 'inputaddrtype']] = baseline[['liennewesttype', 'judgmentnewesttype', 'inputaddrtype']].astype(str)
test_1[['liennewesttype', 'judgmentnewesttype', 'inputaddrtype']] = test_1[['liennewesttype', 'judgmentnewesttype', 'inputaddrtype']].astype(str)
test_2[['liennewesttype', 'judgmentnewesttype', 'inputaddrtype']] = test_2[['liennewesttype', 'judgmentnewesttype', 'inputaddrtype']].astype(str)

In [7]:
bs_seleid = baseline[["id", "seleid"]]
bs_seleid.columns = ["id", "bs_seleid"]
test_1 = test_1.merge(bs_seleid, on = "id")
print(test_1.shape)
test_1 = test_1[test_1["seleid"] == test_1["bs_seleid"]]
print(test_1.shape)

test_2 = test_2.merge(bs_seleid, on = "id")
print(test_2.shape)
test_2 = test_2[test_2["seleid"] == test_2["bs_seleid"]]
print(test_2.shape)

(99937, 2306)
(99937, 2306)
(99939, 2306)
(99939, 2306)


In [39]:
99579 - 98777

802

In [31]:
baseline.select_dtypes(include=[object]).columns

Index(['accountnumber', 'bus_company_name', 'outbestbusname',
       'outbestbusstreetaddr', 'outbestbuscity', 'outbestbusstate',
       'liennewesttype', 'judgmentnewesttype', 'inputaddrtype', 'model1name',
       'model1rc1', 'model1rc2', 'model1rc3', 'model1rc4', 'model2name',
       'model2rc1', 'model2rc2', 'model2rc3', 'model2rc4', 'errorcode', 'id'],
      dtype='object')

In [8]:
from dsgtools.reporting import ImpactAnalysisReport
fmt_score = make_format(cuts = [-np.inf, 500, 550, 600, 625, 650, 675, 700, 725, 750, 775, 800, 825, 850, 900, np.inf], exceptions = [0, 100, 200, 222])
cr = ImpactAnalysisReport(baseline, test_1, keys="id",
           score_columns = ["model1score", "model2score", "sbfehitindex"], 
           score_formats = {"model1score" : fmt_score, 
                         'model2score': fmt_score, },
           exceptions=[-1, -99, -98, -97, 0, 100, 200, 222],
           dataframe_names = ("baseline", "test1"))
cr.to_xlsx("./_temp/air937_202301_test1_data-comparison.xlsx", overwrite= True)

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [9]:
fmt_score = make_format(cuts = [-np.inf, 500, 550, 600, 625, 650, 675, 700, 725, 750, 775, 800, 825, 850, 900, np.inf], exceptions = [0, 100, 200, 222])
cr = ImpactAnalysisReport(baseline, test_2, keys="id",
           score_columns = ["model1score", "model2score", "sbfehitindex"], 
           score_formats = {"model1score" : fmt_score, 
                         'model2score': fmt_score, },
           exceptions=[-1, -99, -98, -97, 0, 100, 200, 222], 
           dataframe_names = ("baseline", "test2"))
cr.to_xlsx("./_temp/air937_202301_test2_data-comparison.xlsx", overwrite= True)

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>